In [ ]:
import torch
import torch.nn as nn
import tiktoken

In [ ]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [ ]:
class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # Use a placeholder for TransformerBlock
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        # Use a placeholder for LayerNorm
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits


class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # A simple placeholder

    def forward(self, x):
        # This block does nothing and just returns its input.
        return x


class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
        # The parameters here are just to mimic the LayerNorm interface.

    def forward(self, x):
        # This layer does nothing and just returns its input.
        return x

In [ ]:
# example of unpacking:
primes = [2, 3, 5, 7, 11]
print(*primes)

2 3 5 7 11


In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [ ]:
model = DummyGPTModel(GPT_CONFIG_124M)

logits = model(batch)
print("Output shape:", logits.shape)
print(logits)

Output shape: torch.Size([2, 4, 50257])
tensor([[[ 0.9785, -0.6872, -0.2119,  ..., -0.3373,  0.4892,  0.6184],
         [ 0.4918,  0.6098, -0.6546,  ..., -0.0368,  0.3166,  0.3460],
         [ 0.5514, -0.0967,  1.9582,  ..., -0.5101, -0.2887, -1.2368],
         [ 1.3463, -1.1147, -1.6701,  ...,  0.3945,  1.2795,  0.6850]],

        [[ 1.2980, -0.9072,  0.3520,  ...,  0.1812,  0.0499,  0.8726],
         [-0.4481,  1.4799, -1.4099,  ..., -1.0378,  1.0683,  0.7372],
         [ 0.4373,  0.1332,  0.9835,  ..., -0.6649, -0.1740, -0.8753],
         [ 0.3907, -1.2496, -1.5639,  ...,  1.3491,  1.7202,  0.4680]]],
       grad_fn=<UnsafeViewBackward0>)
